In [1]:
# Some part of the code was referenced from below.
# https://github.com/pytorch/examples/tree/master/word_language_model 
import torch 
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
from data_utils import Dictionary, Corpus

In [2]:
# Hyper Parameters
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 5
num_samples = 1000   # number of words to be sampled
batch_size = 20
seq_length = 30
learning_rate = 0.002

In [11]:
# Load Penn Treebank Dataset
train_path = './data/train.txt'
sample_path = './sample.txt'
corpus = Corpus()
ids = corpus.get_data(train_path, batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

In [12]:
# RNN Based Language Model
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.init_weights()

    def init_weights(self):
        self.embed.weight.data.uniform_(-0.1, 0.1)
        self.linear.bias.data.fill_(0)
        self.linear.weight.data.uniform_(-0.1, 0.1)

    def forward(self, x, h):
        # Embed word ids to vectors
        x = self.embed(x) 

        # Forward propagate RNN  
        out, h = self.lstm(x, h)

        # Reshape output to (batch_size*sequence_length, hidden_size)
        out = out.contiguous().view(out.size(0)*out.size(1), out.size(2))

        # Decode hidden states of all time step
        out = self.linear(out)  
        return out, h

In [13]:
model = RNNLM(vocab_size, embed_size, hidden_size, num_layers)
model.cuda()

RNNLM(
  (embed): Embedding(10000, 128)
  (lstm): LSTM(128, 1024, batch_first=True)
  (linear): Linear(in_features=1024, out_features=10000, bias=True)
)

In [14]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
# Truncated Backpropagation 
def detach(states):
    return [state.detach() for state in states] 

In [20]:
# Training
for epoch in range(num_epochs):
    # Initial hidden and memory states
    device = torch.device('cuda')
    states = (torch.zeros(num_layers, batch_size, hidden_size, device=device),
              torch.zeros(num_layers, batch_size, hidden_size, device=device))

    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get batch inputs and targets
        inputs = ids[:, i:i+seq_length].cuda()
        targets = ids[:, (i+1):(i+1)+seq_length].contiguous().cuda()

        # Forward + Backward + Optimize
        model.zero_grad()
        states = detach(states)
        outputs, states = model(inputs, states) 
        loss = criterion(outputs, targets.view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        step = (i+1) // seq_length
        if step % 100 == 0:
            print ('Epoch [%d/%d], Step[%d/%d], Loss: %.3f, Perplexity: %5.2f' %
                   (epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))

Epoch [1/5], Step[0/1549], Loss: 3.366, Perplexity: 28.96
Epoch [1/5], Step[100/1549], Loss: 2.934, Perplexity: 18.80
Epoch [1/5], Step[200/1549], Loss: 3.104, Perplexity: 22.29
Epoch [1/5], Step[300/1549], Loss: 3.105, Perplexity: 22.31
Epoch [1/5], Step[400/1549], Loss: 3.024, Perplexity: 20.58
Epoch [1/5], Step[500/1549], Loss: 2.551, Perplexity: 12.82
Epoch [1/5], Step[600/1549], Loss: 3.102, Perplexity: 22.23
Epoch [1/5], Step[700/1549], Loss: 3.039, Perplexity: 20.88
Epoch [1/5], Step[800/1549], Loss: 3.133, Perplexity: 22.93
Epoch [1/5], Step[900/1549], Loss: 2.749, Perplexity: 15.63
Epoch [1/5], Step[1000/1549], Loss: 2.903, Perplexity: 18.23
Epoch [1/5], Step[1100/1549], Loss: 3.026, Perplexity: 20.62
Epoch [1/5], Step[1200/1549], Loss: 3.107, Perplexity: 22.36
Epoch [1/5], Step[1300/1549], Loss: 2.793, Perplexity: 16.33
Epoch [1/5], Step[1400/1549], Loss: 2.511, Perplexity: 12.32
Epoch [1/5], Step[1500/1549], Loss: 2.926, Perplexity: 18.65
Epoch [2/5], Step[0/1549], Loss: 3.0

In [17]:
# Sampling
with open(sample_path, 'w') as f:
    # Set intial hidden ane memory states
    state = (Variable(torch.zeros(num_layers, 1, hidden_size)).cuda(),
             Variable(torch.zeros(num_layers, 1, hidden_size)).cuda())

    # Select one word id randomly
    prob = torch.ones(vocab_size)
    input = Variable(
        torch.multinomial(prob, num_samples=1).unsqueeze(1),
        volatile=True).cuda()

    for i in range(num_samples):
        # Forward propagate rnn
        output, state = model(input, state)

        # Sample a word id
        prob = output.squeeze().data.exp().cpu()
        word_id = torch.multinomial(prob, 1)[0]

        # Feed sampled word id to next time step
        input.data.fill_(word_id)

        # File write
        word = corpus.dictionary.idx2word[word_id.item()]
        word = '\n' if word == '<eos>' else word + ' '
        f.write(word)

        if (i + 1) % 100 == 0:
            print('Sampled [%d/%d] words and save to %s' % (i + 1, num_samples,
                                                            sample_path))

/home/wanglijun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.


Sampled [100/1000] words and save to ./sample.txt
Sampled [200/1000] words and save to ./sample.txt
Sampled [300/1000] words and save to ./sample.txt
Sampled [400/1000] words and save to ./sample.txt
Sampled [500/1000] words and save to ./sample.txt
Sampled [600/1000] words and save to ./sample.txt
Sampled [700/1000] words and save to ./sample.txt
Sampled [800/1000] words and save to ./sample.txt
Sampled [900/1000] words and save to ./sample.txt
Sampled [1000/1000] words and save to ./sample.txt


In [18]:
# Save the Trained Model
torch.save(model.state_dict(), 'model.pkl')